In [1]:
import ChessGame
import numpy as np

from gym import spaces
from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector

from pettingzoo.test import api_test
from pettingzoo.test import performance_benchmark

import ray
import torch

from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork
from ray.rllib.utils.torch_ops import FLOAT_MIN, FLOAT_MAX
from ray.rllib.utils.framework import try_import_tf, try_import_torch

from ray.rllib.agents.callbacks import DefaultCallbacks
# from ray.rllib.env import BaseEnv
# from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
# from ray.rllib.policy import Policy
# from ray.rllib.policy.sample_batch import SampleBatch

tf1, tf, tfv = try_import_tf()
torch, nn = try_import_torch()

class ChessEnv (AECEnv):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        super().__init__()
        # White is represented as player_0 and black is represented as player_1
        self.agents = ["player_" + str(num) for num in range (2)]
        self.possible_agents = ["player_" + str(num) for num in range (2)]
        self.agent_name_mapping = { "player_0" : 0,
                                    "player_1" : 1}
        self._agent_selector = agent_selector(self.agents)

        self.action_spaces = {name: spaces.Discrete(4672) for name in self.possible_agents}
        self.observation_spaces = {name: spaces.Dict({
            'observation': spaces.Box(low=-1, high=500, shape=(5, 8, 8, 14), dtype=np.float32),
            'action_mask': spaces.Box(low=0, high=1, shape=(4672,), dtype=np.float32)
        }) for name in self.possible_agents}

        self.blackEval = 0.5
        self.whiteEval = 0.5
        self.board = ChessGame.Board()
        self.rewards = None
        self.dones = None
        self.infos = {name: {} for name in self.agents}

        self.agent_selection = None

        
    def observe(self, agent):
        observation = []
        action_mask = np.zeros(shape = 4672, dtype = np.int16)

        if (self.board.whiteToMove):
            observation = self.board.GetBoard()[0]
            if (agent == self.possible_agents[0]):
                for moves in self.board.allLegalMoves:
                    action_mask[moves] = 1
        else:
            observation = self.board.GetBoard()[1]
            if (agent == self.possible_agents[1]):
                for moves in self.board.allLegalMoves:
                    action_mask[moves] = 1
        return {"observation" : observation, "action_mask" : action_mask}

    def reset (self):
        self.board.ResetBoard()

        self.agents = self.possible_agents[:]
        self.rewards = {agent : 0 for agent in self.agents}
        self._cumulative_rewards =  {agent : 0 for agent in self.agents}
        self.dones = {agent: False for agent in self.agents}
        self.observation = {agent : None for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}

        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.reset()
        

    def step (self, action):
        if self.dones[self.agent_selection]:
            return self._was_done_step(action)
        
        self.rewards = {agent : 0 for agent in self.agents}
        current_agent = self.agent_selection
        self._cumulative_rewards[current_agent] = 0

        # Make the move
        self.board.Move(action)

        if (self.board.whiteToMove):
            # If it's white to move and there's checkmate on the board, white lost (and black won); apply appropriate rewards
            if (self.board.gameState > 0):
                self.rewards[self.agents[0]] = -1
                self.rewards[self.agents[1]] = 1
            # Game is drawn somehow, each agent gets a reward of 0
            elif (self.board.gameState < 0):
                self.rewards[self.agents[0]] = self.rewards[self.agents[1]] = 0
            else:
                self.rewards[self.agents[0]] = (self.board.eval - self.whiteEval) * 5
                self.whiteEval = self.board.eval

        else:
            # If it's black to move and there's checkmate on the board, black lost (and white won); apply appropriate rewards
            if (self.board.gameState > 0):
                self.rewards[self.agents[0]] = 1
                self.rewards[self.agents[1]] = -1
            # Game is drawn somehow, each agent gets a reward of 0
            elif (self.board.gameState < 0):
                self.rewards[self.agents[0]] = self.rewards[self.agents[1]] = 0
            else:
                self.rewards[self.agents[1]] = (self.board.eval - self.blackEval) * 5
                self.blackEval = self.board.eval

        if (self.board.gameState != 0):
            for name in self.agents:
                self.infos[name] = self.board.listOfMoves
                self.dones[name] = True

        self.agent_selection = self._agent_selector.next()
        self._accumulate_rewards()
 
    def render (self):
        self.board.ShowBoard()
    
    def close(self):
        pass

# Formal PettingZoo API Test & Performance Benchmarks
env = ChessEnv()
# env = wrappers.BaseWrapper(env)
# env = wrappers.OrderEnforcingWrapper(env)
api_test(env, num_cycles = 50, verbose_progress = True)
performance_benchmark(env)

class ChessNetwork (TorchModelV2, nn.Module):
    def __init__ (self, obs_space, action_space, num_outputs, model_config, name, **kwargs):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name, **kwargs)
        nn.Module.__init__(self)

        action_embed_size = 4672
        self.action_embed_model = FullyConnectedNetwork(
            spaces.Box(low=-1, high=500, shape=(5, 8, 8, 14)), action_space, action_embed_size,
            model_config, name + "action_embed")

    def forward(self, input_dict, state, seq_lens):
        # Extract the available actions tensor from the observation.
        action_mask = input_dict["obs"]["action_mask"]

        # Compute the predicted action embedding
        action_logits, _ = self.action_embed_model({
            "obs": input_dict["obs"]['observation']
        })
        
        # Masks out invalid actions
        inf_mask = torch.clamp(torch.log(action_mask), FLOAT_MIN, FLOAT_MAX)

        return action_logits + inf_mask, state

    def value_function(self):
        return self.action_embed_model.value_function()

class AlgebraicNotation (DefaultCallbacks):
    def __init__(self):
        super().__init__()
        # Counter variable of number of total episodes
        self.episode_num = 1
        # Log chess games (algebraic notation) every "log_freq" games
        self.log_freq = 25
    def on_episode_end (self, *, worker, base_env, episode, env_index, **kwargs):
        if (self.episode_num % self.log_freq == 0):
            board = base_env.get_unwrapped()[0].env.board
            gameFile = open("C:/Users/408aa/Desktop/Code/Python/Chess_AI/AlgebraicNotations.txt", "a")
            algNotation = str()

            for turn in board.listOfMoves:
                algNotation = " ".join([algNotation, turn])
            
            algNotation = "".join(["Game #", str(self.episode_num), ": ", algNotation, "\n", "\n"])
            # print (algNotation)
            gameFile.write(algNotation)
            gameFile.close()
        self.episode_num +=1

def env_creator():
    env = ChessEnv()
    return env

# Clear the file with the game logs
def clearLogFile():
    gameFile = open("C:/Users/408aa/Desktop/Code/Python/Chess_AI/AlgebraicNotations.txt", "w")
    gameFile.close()


In [3]:
from ray.rllib.agents import ppo
from ray.rllib.env import PettingZooEnv
from ray import tune

from ray.rllib.models import ModelCatalog
from ray.tune.registry import register_env
# from ray.rllib.agents.registry import get_trainer_class

# from ray.tune.logger import pretty_print

import os


# Register Model and Environment
ModelCatalog.register_custom_model("ChessNetwork", ChessNetwork)

register_env("ChessEnv", lambda config : PettingZooEnv(env_creator()))

test_env = PettingZooEnv(env_creator())
obs_space = test_env.observation_space
act_space = test_env.action_space

config = ppo.DEFAULT_CONFIG.copy()

config["multiagent"] = {
    "policies": {
        "player_0": (None, obs_space, act_space, {}),
        "player_1": (None, obs_space, act_space, {}),
    },
    "policy_mapping_fn": lambda agent_id: agent_id
}

config["num_workers"] = 5
config["num_envs_per_worker"] = 1
config["num_cpus_per_worker"] = 2
config["num_cpus_for_driver"] = 4
# config["num_gpus"] = 1
config["framework"] = "torch"
config["model"] = {
    "custom_model" : "ChessNetwork",
}
config["env"] = "ChessEnv"
config["horizon"] = 150
config["rollout_fragment_length"] = 500
config["callbacks"] = AlgebraicNotation
config["log_level"] = "INFO"
ray.init(num_cpus = 16, num_gpus = 1, ignore_reinit_error = True)

tune.run(
    "PPO",
    name="Chess_Policy",
    stop={"timesteps_total": 750000},
    checkpoint_freq=1000,
    checkpoint_at_end = True,
    config=config,
    local_dir = os.getcwd()
)

ray.shutdown()

2021-07-26 01:23:55,684	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc
PPO_ChessEnv_d8249_00000,PENDING,


2021-07-26 01:24:07,450	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
(pid=5700) 2021-07-26 01:24:13,071	WARNING ppo.py:143 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=5 num_envs_per_worker=1 rollout_fragment_length=500)! Auto-adjusting `rollout_fragment_length` to 800.
(pid=9864) 2021-07-26 01:24:22,558	INFO catalog.py:414 -- Wrapping <class '__main__.ChessNetwork'> as None
(pid=15516) 2021-07-26 01:24:22,681	INFO catalog.py:414 -- Wrapping <class '__main__.ChessNetwork'> as None
(pid=9864) 2021-07-26 01:24:22,748	INFO torch_policy.py:137 -- TorchPolicy (worker=1) running on CPU.
(pid=15516) 2021-07-26 01:24:22,769	INFO torch_policy.py:137 -- TorchPolicy (worker=3) running on CPU.
(pid=9864) 2021-07-26 01:24:22,983	INFO catalog.py:414 -- Wrapping <class '__main__.ChessNetwork'> as None
(pid=15516) 2021-07-26 01:24:22,983	INFO catalog.py:414 -- Wrapping <class '__main__.ChessNetwork'> as None
(pid=16724) 2021-07-26 01:24:22,983

Result for PPO_ChessEnv_d8249_00000:
  agent_timesteps_total: 3975
  custom_metrics: {}
  date: 2021-07-26_01-30-50
  done: false
  episode_len_mean: 170.0
  episode_media: {}
  episode_reward_max: 0.16015395522117615
  episode_reward_mean: 0.006650244817137718
  episode_reward_min: -0.05490392446517944
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: 0bb2662462f74b3ab6e6a400b1419099
  hostname: Asus-ZephryusG14
  info:
    learner:
      player_0:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.2
          cur_lr: 5.0e-05
          entropy: 3.180171713232994
          entropy_coeff: 0.0
          kl: 0.024934864486567676
          policy_loss: 0.22749220672994852
          total_loss: 0.29122585244476795
          vf_explained_var: 0.8705475330352783
          vf_loss: 0.05874667107127607
      player_1:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.2
          cur_lr: 5.0e-05
          entropy: 3.2271

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ChessEnv_d8249_00000,RUNNING,10.0.0.37:5700,1,386.991,4000,0.00665024,0.160154,-0.0549039,170


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ChessEnv_d8249_00000,RUNNING,10.0.0.37:5700,72,27390.6,288000,0.00266837,0.503004,-0.507612,168.97


(pid=16724) C:\Users\408aa\Anaconda3\envs\rl_env\lib\site-packages\ray\rllib\evaluation\collectors\simple_list_collector.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
(pid=16724)   arr = np.array(v)
(pid=16724) C:\Users\408aa\Anaconda3\envs\rl_env\lib\site-packages\ray\rllib\policy\sample_batch.py:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
(pid=16724)   self[k] = np.array(v)


Result for PPO_ChessEnv_d8249_00000:
  agent_timesteps_total: 290331
  custom_metrics: {}
  date: 2021-07-26_09-07-27
  done: false
  episode_len_mean: 168.71
  episode_media: {}
  episode_reward_max: 0.5030038952827454
  episode_reward_mean: -0.0016481637954711913
  episode_reward_min: -0.5076117813587189
  episodes_this_iter: 25
  episodes_total: 1752
  experiment_id: 0bb2662462f74b3ab6e6a400b1419099
  hostname: Asus-ZephryusG14
  info:
    learner:
      player_0:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 1.0125000000000002
          cur_lr: 5.0e-05
          entropy: 0.5815132036805153
          entropy_coeff: 0.0
          kl: 0.011957329872529954
          policy_loss: 0.007326827268116176
          total_loss: 0.06937311892397702
          vf_explained_var: 0.6833558082580566
          vf_loss: 0.04993949248455465
      player_1:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 1.0125000000000002
          cur_

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ChessEnv_d8249_00000,RUNNING,10.0.0.37:5700,73,27779.4,292000,-0.00164816,0.503004,-0.507612,168.71


2021-07-26 09:10:00,211	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 
